In [57]:
import torch
import models,settings
import numpy as np

In [58]:
ckpt = torch.load(f"../{settings.MODEL}", map_location="cpu")
clf = models.BowmanEntailmentClassifier
enc = models.TextEncoder(len(ckpt["stoi"]))
model=clf(enc)
model.load_state_dict(ckpt['state_dict'])

<All keys matched successfully>

In [40]:
n=torch.randn(2,4)
print(n)
def prune_by_percent_once(percent, mask, final_weight):
    # Put the weights that aren't masked out in sorted order.
    sorted_weights = np.sort(np.abs(final_weight[mask != 0]))

    # Determine the cutoff for weights to be pruned.
    
    cutoff_index = np.round(percent * sorted_weights.size).astype(int)
    cutoff = sorted_weights[cutoff_index - 1] 
    # Prune all weights below the cutoff.
    new_mask= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), mask)
    
    new_weights= torch.where(torch.abs(torch.tensor(final_weight)) <= cutoff, torch.zeros(mask.shape), torch.tensor(final_weight))
    return new_mask, new_weights
prune_by_percent_once(0.5, torch.ones(weights.shape),weights) 

tensor([[-0.6631,  1.0282,  0.0766, -0.3954],
        [-1.7383, -0.3737, -0.1654,  1.2035]])


(tensor([[1., 0., 1.,  ..., 0., 0., 1.],
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 1., 0., 0.],
         ...,
         [1., 0., 1.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 1., 0., 1.],
         [1., 0., 0.,  ..., 0., 1., 1.]]),
 tensor([[ 0.1021,  0.0000, -0.1801,  ...,  0.0000,  0.0000,  0.1054],
         [ 0.1023,  0.0000,  0.1012,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000, -0.0737,  0.1627,  ..., -0.0726,  0.0000,  0.0000],
         ...,
         [ 0.2561,  0.0000,  0.0704,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000, -0.1102,  ..., -0.1076,  0.0000, -0.4026],
         [ 0.1069,  0.0000,  0.0000,  ...,  0.0000, -0.1041, -0.0549]]))

In [59]:
prune_metrics_dir = "../models/snli/Inc/prune_metrics/0.5%Pruned/model_best.pth"
weights=torch.load(f"{prune_metrics_dir}")['state_dict']
model.load_state_dict(weights)

<All keys matched successfully>

In [60]:
a=torch.where(model.mlp[0].weight==0,1,0).sum()/100
a.item()

54.54999923706055

In [61]:
weights = model.mlp[0].weight.t().detach().cpu().numpy()

In [63]:
nm,nw=prune_by_percent_once(0.5, torch.ones(weights.shape),weights) 

In [130]:
mask=torch.ones(weights.shape)
final_weight=np.sort(np.abs(weights>0)).astype('int')

In [64]:
model.mlp[0].weight.t().detach().cpu().copy(nw)

tensor([[ 0.1409,  0.0000, -0.1801,  ...,  0.0000,  0.0000,  0.1419],
        [ 0.0905,  0.0000,  0.1012,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0737,  0.1627,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.2711,  0.0000,  0.0704,  ...,  0.0000,  0.0000, -0.0821],
        [ 0.0000,  0.0000, -0.1102,  ..., -0.2112,  0.0000, -0.4373],
        [ 0.1016,  0.0000,  0.0000,  ..., -0.0884, -0.1041,  0.0000]])

In [55]:
a=torch.where(torch.abs(torch.tensor(final_weight)) == 1, torch.zeros(mask.shape), mask)

NameError: name 'final_weight' is not defined

In [65]:
torch.equal(model.mlp[0].weight.t().detach().cpu(),nw)

True

In [129]:
np.round((torch.where(model.mlp[0].weight.t()==0,1,0).sum()/(1024*2048)),1)

tensor(0.5000)